In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet -U automl_alex

In [1]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.07.18


In [3]:
RANDOM_SEED = 42

# Classifier

## Data

In [2]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((750, 20), (250, 20))

## Model

In [5]:
model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED, verbose=1)

Source X_train shape:  (750, 20) | X_test shape:  (250, 20)
##################################################
Auto detect cat features:  13
> Start preprocessing Data
> Generate cat encodet features
 +  55  Features from  OneHotEncoder
 +  44  Features from  HelmertEncoder
 +  54  Features from  HashingEncoder
 +  16  Features from  FrequencyEncoder
> Generate Frequency Encode num features
 +  4  Frequency Encode Num Features 
> Clean Nans in num features
> Generate interaction Num Features
 +  24  Interaction Features
> Normalization Features
##################################################
> Total Features:  201
##################################################
New X_train shape:  (750, 201) | X_test shape:  (250, 201)


In [6]:
%%time
predict_test, predict_train = model.fit_predict(timeout=1000, verbose=2)


 Opt BestModels
One iteration takes ~ 0.6 sec
> Start Auto calibration parameters
> Start optimization with the parameters:
CV_Folds =  10
Score_CV_Folds =  3
Feature_Selection =  True
Opt_lvl =  2
Cold_start =  46.0
Early_stoping =  100
Metric =  roc_auc_score
Direction =  maximize
##################################################
Default model OptScore = 0.5588
Optimize: : 88it [06:43,  4.59s/it,  | Model: CatBoost | OptScore: 0.7979 | Best roc_auc_score: 0.827 +- 0.029059]

 Predict from Models_1
 10%|█         | 1/10 [01:20<12:07, 80.83s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7927 std: 0.050418
 20%|██        | 2/10 [02:42<10:49, 81.13s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7898 std: 0.04651
 30%|███       | 3/10 [03:50<08:59, 77.09s/it]
 Mean Score roc_auc_score on 30 Folds: 0.7917 std: 0.049745
 40%|████      | 4/10 [05:26<08:16, 82.81s/it]
 Mean Score roc_auc_score on 30 Folds: 0.795 std: 0.042919
 50%|█████     | 5/10 [06:45<06:49, 81.82s/it]
 Mean Score roc_auc_

In [7]:
predict_test[:5]

array([0.50593874, 0.47516297, 0.62275006, 0.4502443 , 0.2075947 ])

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:  0.8191


In [9]:
model.stack_models_cfgs

,score_opt,model_score,score_std,model_name,model_param,wrapper_params,cat_encoders,columns,cv_folds
0,0.7979,0.8270,0.029059,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
1,0.7830,0.8157,0.032703,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
2,0.7810,0.8206,0.039552,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_property_magnitude_2, OneH...",10
3,0.7759,0.8064,0.030494,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
4,0.7727,0.8124,0.039652,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
5,0.7657,0.8142,0.048497,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
6,0.7628,0.8025,0.039686,KNeighbors,"{'n_jobs': -1, 'n_neighbors': 59, 'weights': '...",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[duration, age, num_dependents, OneHotEncoder_...",10
7,0.7625,0.8215,0.059007,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[OneHotEncoder_residence_since, OneHotEncoder_...",10
8,0.7613,0.8043,0.043044,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
9,0.7611,0.8154,0.054328,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","[age, OneHotEncoder_residence_since, OneHotEnc...",10
